In [40]:
# Packages Download
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
#import re
#import konlpy
#from konlpy.tag import Okt
#from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
#from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import warnings
import tensorflow as tf

# ignore warnings
#warnings.filterwarnings(action='ignore')
#old_v = tf.logging.get_verbosity()
#tf.logging.set_verbosity(tf.logging.ERROR)

# os set
#os.getcwd()

#########################################################################
##1.	개봉일부터 2주차까지 짜르기
##      개봉일이 없다면 그냥 2주까지만 남기기
##2.	categorical 변수에 대해서, 카테고리가 너무 많다면 cluster로 조금 줄여줘
##3.	numeric 변수에 대해서 고려해야할 방안
##  i.	 correlation을 통해서 중복성있는 데이터 삭제
##  ii.	양으로 할지 증감율로 할지.
##  iii.	Scale 조정 방법
##4.	Na 처리
#########################################################################


# road data
# 원본데이터
movie = pd.read_csv("kobis_join.csv", encoding='UTF-8')
# 14일치 개봉일 잘린데이터
#movie2 = pd.read_csv(R"C:\Users\heon1\Desktop\movie2.csv", encoding='UTF-8')
# 14일치 개봉x까지 잘린 데이터
#movie3 = pd.read_csv(R"C:\Users\heon1\Desktop\movie3.csv", encoding='UTF-8')


# data view
movie.columns
movie.describe()
movie.shape
movie.isna().sum() # 방법은 있는데 필요는 없어
                   # na 관객수 643, 누적관객수 51 - 관객수는 누적- 전날 누적 / 누적관객수는 전날누적 + 당일 관객수
                   # 영화명, 감독, 제작사, 수입사, 배급사, 개봉일, 장르, 관람등급, 영화구분, 외국영화 기본 13516개

# category 정리하기
                    # dummy clustering # not yet
                    # 외국영화, 영화구분, 관람등급은 그냥 가능
                    # 장르부터는 클러스터링 필요

dummycolumns = ['외국영화', '영화구분', '관람등급', '장르', '배급사', '수입사', '제작사', '감독']

for dim in dummycolumns : #개별 숫자까지 보는법
    dups_count2 = movie.pivot_table(index=[dim], aggfunc='size')
    print(dups_count2,"na %9d" %movie[dim].isna().sum(),  sep='\n')

for dim in dummycolumns : #칼럼 별 숫자만 보는법
    dups_count2 = movie.pivot_table(index=[dim], aggfunc='size')
    print(dim , " : ", len(dups_count2))

movie['movie_id.x.i.']

외국영화
0.0    17926
1.0    24196
dtype: int64
na     13516
영화구분
독립/예술영화     1585
일반영화       40537
dtype: int64
na     13516
관람등급
0.0      9041
12.0    12976
15.0    12978
18.0     7127
dtype: int64
na     13516
장르
SF          1020
가족           253
공연          1425
공포(호러)      1367
기타            61
다큐멘터리        573
드라마         9689
멜로/로맨스      3407
뮤지컬          309
미스터리         811
범죄          1925
사극          1235
서부극(웨스턴)       8
성인물(에로)      330
스릴러         1270
애니메이션       8229
액션          6564
어드벤처         516
전쟁           279
코미디         2095
판타지          716
dtype: int64
na     13556
배급사
(유)쏘아필름                           4
(유)영화사 화수분                       65
(유)영화사 화수분,(주)그림상자                1
(유)영화사 화수분,길벗영화(주)                1
(유)영화사 화수분,인트로미디어(주)              1
(유)영화사 화수분,코리아픽쳐스(주)             34
(유)조이앤시네마                        52
(유)조이앤컨텐츠그룹                      81
(재)CBS                           26
(주)가온콘텐츠                          4
(주)그라운드미디어                        1
(주)김기

0        20150976
1        20150976
2        20150976
3        20150976
4        20150976
5        20150976
6        20150976
7        20150976
8        20150976
9        20150976
10       20150976
11       20150976
12       20150976
13       20150976
14       20150976
15       20150976
16       20150976
17       20150976
18       20150976
19       20150976
20       20150976
21       20150976
22       20150976
23       20150976
24       20150976
25       20150976
26       20150976
27       20150976
28       20150976
29       20150976
           ...   
55608    19790068
55609    19790068
55610    19468008
55611    19468008
55612    19468008
55613    19468008
55614    19468008
55615    19468008
55616    19468008
55617    19468008
55618    19468008
55619    19468008
55620    19468008
55621    19468008
55622    19468008
55623    19468008
55624    19468008
55625    20176043
55626    20176043
55627    20176062
55628    20176062
55629    20176042
55630    20176042
55631    20176063
55632    2

In [41]:
## 1. 2주차까지 짜르기
# 각 영화별 수
movie.columns[1]

movie = movie.sort_values(axis=0, by=[movie.columns[1],movie.columns[2]]) # 영화별로 날짜 sorting이 안맞으니까
movie = movie.reset_index() # index 재정렬
movie = movie.drop(['index'], axis= 1)

movie.to_csv("KOBISsample1.csv", encoding='utf-8-sig')
dups_count = movie.pivot_table(index=['movie_id.x.i.'], aggfunc='size')

# 날짜를 date로
from datetime import datetime, timedelta
movie["날짜"] = pd.to_datetime(movie["날짜"], format='%Y-%m-%d')
movie["개봉일"] = pd.to_datetime(movie["개봉일"], format='%Y-%m-%d')

In [42]:
movie

,h_name_2,movie_id.x.i.,날짜,개봉일,스크린수,스크린점유율,상영횟수,상영점유율,좌석수,좌석점유율,...,순위,영화명,감독,제작사,수입사,배급사,장르,관람등급,영화구분,외국영화
0,마지막 웃음,19248002,2017-04-02,NaT,1.0,0.0,1.0,0.0,212,0.0,...,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,마지막 웃음,19248002,2017-04-06,NaT,1.0,0.0,1.0,0.0,212,0.0,...,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,마지막 웃음,19248002,2017-04-18,NaT,1.0,0.0,1.0,0.0,212,0.0,...,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,탐욕,19258006,2017-08-03,2016-05-12,1.0,0.0,1.0,0.0,202,0.0,...,47,탐욕,김정욱,주식회사 루믹스미디어,NaN,주식회사 루믹스미디어,멜로/로맨스,18.0,일반영화,0.0
4,탐욕,19258006,2017-08-16,2016-05-12,1.0,0.0,1.0,0.0,202,0.0,...,61,탐욕,김정욱,주식회사 루믹스미디어,NaN,주식회사 루믹스미디어,멜로/로맨스,18.0,일반영화,0.0
5,10월,19278004,2017-03-08,NaT,1.0,0.0,1.0,0.0,202,0.0,...,63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10월,19278004,2017-03-12,NaT,1.0,0.0,1.0,0.0,202,0.0,...,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,날개,19278015,2017-08-09,NaT,1.0,0.0,1.0,0.0,202,0.0,...,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,날개,19278015,2017-08-24,NaT,1.0,0.0,1.0,0.0,202,0.0,...,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,어셔가의 몰락,19280003,2017-06-18,NaT,1.0,0.0,1.0,0.0,212,0.0,...,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# 개봉일 있을떄
import pandas as pd
openDt = 0
for i in range(len(movie)) :
    if(pd.isna(movie.loc[i, "개봉일"])):#개봉일 없으면
        if(i-1 in movie.index and movie.loc[i, "movie_id.x.i."] == movie.loc[i-1, "movie_id.x.i."]):
            movie.loc[i, '개봉일'] = openDt
            if movie.loc[i, '날짜'] > (movie.loc[i, '개봉일'] + timedelta(days=13) ):
                if(i+1 in movie.index and movie.loc[i, "movie_id.x.i."] == movie.loc[i+1, "movie_id.x.i."]):
                    movie.loc[i+1, "개봉일"] = movie.loc[i, "개봉일"]
                    openDt = movie.loc[i, "개봉일"]
                movie.drop(i, inplace=True)
        else:
            openDt = movie.loc[i, '날짜']
            movie.loc[i, "개봉일"] = openDt
    else:
        if movie.loc[i, '날짜'] > (movie.loc[i, '개봉일'] + timedelta(days=13) ):
            if(i+1 in movie.index and movie.loc[i, "movie_id.x.i."] == movie.loc[i+1, "movie_id.x.i."]):
                    movie.loc[i+1, "개봉일"] = movie.loc[i, "개봉일"]
                    openDt = movie.loc[i, "개봉일"]
            movie.drop(i, inplace=True)

In [44]:
movie

,h_name_2,movie_id.x.i.,날짜,개봉일,스크린수,스크린점유율,상영횟수,상영점유율,좌석수,좌석점유율,...,순위,영화명,감독,제작사,수입사,배급사,장르,관람등급,영화구분,외국영화
0,마지막 웃음,19248002,2017-04-02,2017-04-02,1.0,0.0,1.0,0.0,212,0.0,...,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,마지막 웃음,19248002,2017-04-06,2017-04-02,1.0,0.0,1.0,0.0,212,0.0,...,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10월,19278004,2017-03-08,2017-03-08,1.0,0.0,1.0,0.0,202,0.0,...,63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10월,19278004,2017-03-12,2017-03-08,1.0,0.0,1.0,0.0,202,0.0,...,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,날개,19278015,2017-08-09,2017-08-09,1.0,0.0,1.0,0.0,202,0.0,...,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,어셔가의 몰락,19280003,2017-06-18,2017-06-18,1.0,0.0,1.0,0.0,212,0.0,...,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,어셔가의 몰락,19280003,2017-06-22,2017-06-18,1.0,0.0,1.0,0.0,212,0.0,...,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,어셔가의 몰락,19280003,2017-06-30,2017-06-18,1.0,0.0,1.0,0.0,212,0.0,...,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,잔다르크의 수난,19288001,2015-04-07,2015-04-07,1.0,0.0,6.0,0.0,1800,0.1,...,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,잔다르크의 수난,19288001,2015-04-08,2015-04-07,1.0,0.0,6.0,0.0,1800,0.1,...,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
movie.to_csv("KOBISsample2.csv", encoding='utf-8-sig')#여기까지가 14일 자르기(재개봉 없는)

In [47]:
movie

,h_name_2,movie_id.x.i.,날짜,개봉일,스크린수,스크린점유율,상영횟수,상영점유율,좌석수,좌석점유율,...,순위,영화명,감독,제작사,수입사,배급사,장르,관람등급,영화구분,외국영화
0,마지막 웃음,19248002,2017-04-02,2017-04-02,1.0,0.0,1.0,0.0,212,0.0,...,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,마지막 웃음,19248002,2017-04-06,2017-04-02,1.0,0.0,1.0,0.0,212,0.0,...,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10월,19278004,2017-03-08,2017-03-08,1.0,0.0,1.0,0.0,202,0.0,...,63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10월,19278004,2017-03-12,2017-03-08,1.0,0.0,1.0,0.0,202,0.0,...,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,날개,19278015,2017-08-09,2017-08-09,1.0,0.0,1.0,0.0,202,0.0,...,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,어셔가의 몰락,19280003,2017-06-18,2017-06-18,1.0,0.0,1.0,0.0,212,0.0,...,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,어셔가의 몰락,19280003,2017-06-22,2017-06-18,1.0,0.0,1.0,0.0,212,0.0,...,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,어셔가의 몰락,19280003,2017-06-30,2017-06-18,1.0,0.0,1.0,0.0,212,0.0,...,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,잔다르크의 수난,19288001,2015-04-07,2015-04-07,1.0,0.0,6.0,0.0,1800,0.1,...,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,잔다르크의 수난,19288001,2015-04-08,2015-04-07,1.0,0.0,6.0,0.0,1800,0.1,...,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df = pd.read_csv('KOBISsample2.csv')#파일의 첫번쨰 칼럼 지우고 사용

In [49]:
df["시사회여부"] = np.arange(len(df))
df["시사회여부"] = "0"
df

,h_name_2,movie_id.x.i.,날짜,개봉일,스크린수,스크린점유율,상영횟수,상영점유율,좌석수,좌석점유율,...,영화명,감독,제작사,수입사,배급사,장르,관람등급,영화구분,외국영화,시사회여부
0,마지막 웃음,19248002,2017-04-02,2017-04-02,1.0,0.0,1.0,0.0,212,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,마지막 웃음,19248002,2017-04-06,2017-04-02,1.0,0.0,1.0,0.0,212,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,10월,19278004,2017-03-08,2017-03-08,1.0,0.0,1.0,0.0,202,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,10월,19278004,2017-03-12,2017-03-08,1.0,0.0,1.0,0.0,202,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,날개,19278015,2017-08-09,2017-08-09,1.0,0.0,1.0,0.0,202,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,어셔가의 몰락,19280003,2017-06-18,2017-06-18,1.0,0.0,1.0,0.0,212,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,어셔가의 몰락,19280003,2017-06-22,2017-06-18,1.0,0.0,1.0,0.0,212,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,어셔가의 몰락,19280003,2017-06-30,2017-06-18,1.0,0.0,1.0,0.0,212,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,잔다르크의 수난,19288001,2015-04-07,2015-04-07,1.0,0.0,6.0,0.0,1800,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,잔다르크의 수난,19288001,2015-04-08,2015-04-07,1.0,0.0,6.0,0.0,1800,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [50]:
movieid = 0
count = 0;
for i in range(len(df)):
    if(movieid !=  df.loc[i, "개봉일"]):
        movieid = df.loc[i, "개봉일"]
        if(df.loc[i, "개봉일"] == df.loc[i+1, "개봉일"]):
            if(df.loc[i, "날짜"] == df.loc[i, "개봉일"]):
                count = 1
            else:
                df.drop(i, inplace=True)
                count = 0
    else:
        if(df.loc[i, "날짜"] < df.loc[i, "개봉일"] and df.loc[i, "개봉일"] == df.loc[i+1, "개봉일"]):
            df.loc[i+1, "시사회여부"] ="1"
            df.drop(i, inplace=True)
        else:
            if(df.loc[i, "시사회여부"] == "1" and df.loc[i, "개봉일"] == df.loc[i+1, "개봉일"]):
                df.loc[i+1, "시사회여부"] = "1"
            count = count +1
            if(count != 1 or count != 7 or count != 14):
                df.drop(i, inplace=True)

In [51]:
df

,h_name_2,movie_id.x.i.,날짜,개봉일,스크린수,스크린점유율,상영횟수,상영점유율,좌석수,좌석점유율,...,영화명,감독,제작사,수입사,배급사,장르,관람등급,영화구분,외국영화,시사회여부
0,마지막 웃음,19248002,2017-04-02,2017-04-02,1.0,0.0,1.0,0.0,212,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,10월,19278004,2017-03-08,2017-03-08,1.0,0.0,1.0,0.0,202,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,날개,19278015,2017-08-09,2017-08-09,1.0,0.0,1.0,0.0,202,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,어셔가의 몰락,19280003,2017-06-18,2017-06-18,1.0,0.0,1.0,0.0,212,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,잔다르크의 수난,19288001,2015-04-07,2015-04-07,1.0,0.0,6.0,0.0,1800,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
11,아리랑 2,19300016,2017-10-14,2017-10-14,1.0,0.0,1.0,0.0,125,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
12,모로코,19308006,2012-11-05,2012-11-05,1.0,0.0,4.0,0.0,2332,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
18,캐벌케이드,19338006,2015-06-05,2015-06-05,1.0,0.0,5.0,0.0,1500,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
22,나는 비밀을 알고 있다,19348007,2018-05-18,2018-05-18,1.0,0.0,1.0,0.0,200,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
23,아란 사람,19348009,2017-05-03,2017-05-03,1.0,0.0,1.0,0.0,212,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [52]:
df.to_csv("KOBISsample3.csv", encoding='utf-8-sig')